OLD CODE, from TrentoTreeMap

In [13]:
import pandas as pd
import geopandas as gpd

In [3]:
bologna_trees_url = "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/alberi-manutenzioni/exports/geojson?lang=it&timezone=Europe%2FRome"
statistical_areas_url ="https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/aree-statistiche/exports/shp?lang=it&timezone=Europe%2FRome"
green_areas_url = "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/verde/exports/shp?lang=it&timezone=Europe%2FBerlin"
aria_quality_url = "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/centraline-qualita-aria/exports/shp?lang=it&timezone=Europe%2FBerlin"
trees_under_maintenance_url= "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/alberi-manutenzioni/exports/csv?lang=it&timezone=Europe%2FRome&use_labels=true&delimiter=%3B"

In [4]:
trees_under_maintenance = pd.read_csv(trees_under_maintenance_url, sep=";")

itree_data = pd.read_csv('../data/temp/input_itree.csv', sep=";")
bologna_trees_file = pd.read_csv('../data/bologna_trees.csv', sep=";")
alberi_manutenzioni = pd.read_csv('../data/alberi-manutenzioni.csv', sep=";")

In [16]:
trees_under_maintenance.columns

Index(['Progressivo pianta', 'Codice albero', 'Pianta isolata',
       'Specie arborea', 'Classe di altezza', 'Dimora', 'Irrigazione',
       'Albero di pregio', 'Classe circonferenza (diametro)',
       'Data inserimento inventario', 'Data ultimo aggiornamento',
       'Distanza dai fabbricati', 'Data Impianto', 'Geo Point', 'Geo Shape',
       'Campagna', 'Zona di prossimità', 'Area statistica'],
      dtype='object')

In [14]:
bologna_trees = gpd.read_file(bologna_trees_url,format="GeoJSON")
bologna_trees.shape

(86346, 18)

In [ ]:
input_itree = pd.read_csv('../data/temp/input_itree.csv', sep=";")

In [3]:
# the final data result inserted in the itree eco app
input_itree = pd.read_csv('../data/temp/input_itree.csv', sep=";")
output_file_cleaned = pd.read_excel('../data/old/output_file_cleaned.xlsx')

# excel files, the data by species
benefit_sum_by_species = pd.read_excel('../data/temp/benefit_sum_by_species.xlsx', header=1)
potential_pest_risk = pd.read_excel('../data/temp/potential_pest_risk.xlsx')
leaf_report_by_species = pd.read_excel('../data/temp/leaf_report_by_species.xlsx')
carbon_storage_by_species = pd.read_excel('../data/temp/carbon_storage_by_species.xlsx')

# csv files, the data by tree
benefit_sum_by_tree = pd.read_csv('../data/temp/benefit_sum_by_tree.csv', sep=";")
carbon_storage_by_tree = pd.read_csv('../data/temp/carbon_storage_by_tree.csv', sep=";")
benefit_price = pd.read_csv('../data/temp/benefit_price.csv', sep=";")

In [4]:
leaf_report_by_species.head()

,Stratum,Species,Trees,Unnamed: 3,Leaf Area,Unnamed: 5,Leaf Biomass,Unnamed: 7,Tree Dry Weight Biomass,Unnamed: 9,Average Condition
0,NaN,NaN,Number,SE,(ac),SE,(ton),SE,(ton),SE,(%)
1,Study Area,European silver fir,23,±0,0.607,"±0,000",0.382,"±0,000",15.848,"±0,000",82.5
2,NaN,Spanish fir,4,±0,0.002,"±0,000",0.001,"±0,000",0.025,"±0,000",82.5
3,NaN,Maple spp,32,±0,0.034,"±0,000",0.008,"±0,000",0.185,"±0,000",82.5
4,NaN,Hedge maple,3811,±0,21.809,"±0,000",5.476,"±0,000",431.274,"±0,000",82.5


In [42]:
# merge the benefit_sum_by_tree data with the starting output_file_cleaned data 
# let's merge the two dataframes by the columns 'Tree ID' (benefit_sum_by_tree) and 'Progressivo pianta' (output_file_cleaned)
benefit_sum_by_tree_new = pd.merge(benefit_sum_by_tree, output_file_cleaned, left_on='Tree ID', right_on='Progressivo pianta', how='left')

# merge the benefit_sum_by_tree data with the starting output_file_cleaned data 
# let's merge the two dataframes by the columns 'Tree ID' (benefit_sum_by_tree) and 'Progressivo pianta' (output_file_cleaned)
benefit_sum_by_tree_new = pd.merge(benefit_sum_by_tree_new, input_itree, left_on='Tree ID', right_on='ID', how='left')


0

In [43]:
benefit_sum_by_tree_new.columns

Index(['Tree ID', 'Species Name', 'DBH (in)', 'Replacement Value (€)',
       'Carbon Storage (lb)', 'Carbon Storage (€)',
       'Gross Carbon Sequestration (lb/yr)',
       'Gross Carbon Sequestration (€/yr)', 'Avoided Runoff (gal/yr)',
       'Avoided Runoff (€/yr)', 'Carbon Avoided (lb/yr)',
       'Carbon Avoided (€/yr)', 'Pollution Removal (oz/yr)',
       'Pollution Removal (€/yr)', 'Energy Savings (€/yr)',
       'Total Annual Benefits (€/yr)', 'Progressivo pianta', 'Codice albero',
       'Pianta isolata', 'Specie arborea', 'Classe di altezza', 'Dimora',
       'Irrigazione', 'Albero di pregio', 'Classe circonferenza (diametro)',
       'Data inserimento inventario', 'Data ultimo aggiornamento',
       'Distanza dai fabbricati', 'Data Impianto', 'Geo Point', 'Geo Shape',
       'Campagna', 'Zona di prossimità', 'Area statistica', 'dbh',
       'total_height', 'Latitude', 'Longitude'],
      dtype='object')

In [23]:
# find the length of each file
rep_6_len = len(rep_6)
rep_6_len

64715

In [19]:
# Clean the first report: benefit_sum_by_species

# Drop the empty first column
benefit_sum_by_species = benefit_sum_by_species.iloc[:, 1:]

# Drop the first row (contains the units)
benefit_sum_by_species = benefit_sum_by_species.iloc[1:, :]

# Rename the columns to reinsert the units: 
    # Carbon Storage -> Carbon Storage (ton)
    # Unnamed: 4 -> Carbon Storage (€)
    # Gross Carbon Sequestration -> Gross Carbon Sequestration (ton/yr)
    # Unnamed: 6 -> Gross Carbon Sequestration (€/yr)
    # Avoided Runoff -> Avoided Runoff (gal/yr)
    # Unnamed: 8 -> Avoided Runoff (€/yr)
    # Pollution Removal -> Pollution Removal (ton/yr)
    # Unnamed: 10 -> Pollution Removal (€/yr)

benefit_sum_by_species = benefit_sum_by_species.rename(columns={'Carbon Storage': 'Carbon Storage (ton)', 'Unnamed: 4': 'Carbon Storage (€)', 'Gross Carbon Sequestration': 'Gross Carbon Sequestration (ton/yr)', 'Unnamed: 6': 'Gross Carbon Sequestration (€/yr)', 'Avoided Runoff': 'Avoided Runoff (gal/yr)', 'Unnamed: 8': 'Avoided Runoff (€/yr)', 'Pollution Removal': 'Pollution Removal (ton/yr)', 'Unnamed: 10': 'Pollution Removal (€/yr)'})

benefit_sum_by_species.head()

,Species,Trees,Carbon Storage (ton),Carbon Storage (€),Gross Carbon Sequestration (ton/yr),Gross Carbon Sequestration (€/yr),Avoided Runoff (gal/yr),Avoided Runoff (€/yr),Pollution Removal (ton/yr),Pollution Removal (€/yr),Replacement Value
1,European silver fir,23,7.92,1154.93,0.18,26.76,79.7,0.57,0,42.37,60479.6
2,Spanish fir,4,0.01,1.81,0,0.39,0.27,0,0,0.14,229.79
3,Maple spp,32,0.09,13.45,0.03,4.19,4.4,0.03,0,2.34,2265.39
4,Hedge maple,3811,215.64,31430.16,6.93,1010.5,2862.29,20.61,0.14,1521.78,1581745.03
5,Pere David's Maple,2,0,0.53,0,0.21,0.19,0,0,0.1,109.64
...,...,...,...,...,...,...,...,...,...,...,...
215,Schneider zelkova,23,0.15,22.5,0.02,3.27,3.94,0.03,0,2.09,2355.77
216,Japanese zelkova,254,0.71,103.52,0.17,24.92,35.7,0.26,0,18.98,22392.86
217,Common jujube,5,0.05,7.63,0.01,1.39,0.57,0,0,0.3,701.71
218,Total,64715,14750.02,2149887.06,491.58,71650.55,209077.45,1505.36,10.48,111158.93,88925289.83
